# Momentum Strategy
This notebook allows automizes investing using a Momentum Strategy, looking at 4 different timeslots (1 month, 3 months, 6 months and a year) to give an optimal recommendation of how to invest, outputting the entire list to a Excel format, as well as plotting the resulting breakdown at the end.

Execute all the cells one after another.

In [ ]:
import matplotlib.pyplot as plt
import yfinance as yf
import datetime as dt

import time

import multiprocessing as mp

import xlsxwriter

import platform

import WebScraper
import API
from main import *

# Momentum Strategy
The change of value from the different timeslots are denoted by $\mu_i, 1 \leq i \leq 4$, called the momenta. They are combined into a single quantity
$$
\mu := \frac{\sum_i w_i \mu_i}{\sum_i w_i}
$$
called the quality momentum. Here $w_i \geq 0, 1 \leq i \leq 4$ are non-negative numbers called weights. They can be freely chosen, and they determine the strategy by *weighting* how much recent growth is valued. The larger the relative value of one of the weights the more impactful the corresponding momentum. The standard weight choice is $(w_1, w_2, w_3, w_4) = (1, 2, 3, 4)$.

Please enter how much capital you want to invest, as well as the weights (note that they can't be negative and at least one has to be non-zero, can leave them blank to get standard values):

In [ ]:
capital_string = input("How many money do you want to invest?\nCapital = ")
if(capital_string == ""):
    capital_string = "1000000"

try:
    capital = float(capital_string)
except:
    raise TypeError("Please enter a valid number.")

if(capital <= 0):
    raise ValueError("The capital has to be positive.")

w1_string = input("Weight 1: ")
if(w1_string == ""):
    w1_string = "1"
w2_string = input("Weight 2: ")
if(w2_string == ""):
    w2_string = "2"
w3_string = input("Weight 3: ")
if(w3_string == ""):
    w3_string = "3"
w4_string = input("Weight 4: ")
if(w4_string == ""):
    w4_string = "4"
    
if(w1_string == ""):
    w1_string = "1"
try:
    w1 = float(w1_string)
    w2 = float(w2_string)
    w3 = float(w3_string)
    w4 = float(w4_string)
except:
    raise TypeError("The weights have to be valid numbers.")
    
if(w1 < 0 or w2 < 0 or w3 < 0 or w4 < 0):
    raise ValueError("The weights have to be non-negative.")

if(w1 > 10000 or w2 > 10000 or w3 > 10000 or w4 > 10000):
    raise ValueError("The weights can't be that larger than 10000.")
    
if(w1 + w2 + w3 + w4 == 0):
    raise ValueError("At least one of the weights has to be non-zero.")
    
totalWeight = w1 + w2 + w3 + w4
weights = [w1 / totalWeight, w2 / totalWeight, w3 / totalWeight, w4 / totalWeight]

print(f"Capital = ${capital:.2f}, The adjusted Weights are {str(weights)}.")

# Api Call
The following cell gets all of the Tikers in the SNP500 (with some unsuitable exceptions) and then calls the Yahoo Finance API to get the corresponding stock values, saving them into a dictionary of dataframes. The API call takes roughly 2 minutes.

In [ ]:
tickers = WebScraper.SNP500Tickers(WriteToFile = True, Sorted = True)

In [ ]:
# Multiprocessing in Jupyter Notebooks does not work on Windows
isMultiprocessing = (platform.system() != "Windows")

if(isMultiprocessing):
    print(f"Multithreading is actived, number of parallel processes is {processCount}")
    df_dict = API.APICallMultiprocessing(tickers = tickers, processCount = 6)
else: # Expected running time is ~3 minutes
    print("Multithreading is deactived, note that this is going to be quite slow (~3min).")
    df_dict = API.APICall(tickers, Output = False, Time = False)

In [ ]:
lastIndex = len(df_dict[tickers[0]].index) - 1

indices = []
n = lastIndex // 12
for i in range(n):
    indices.append(lastIndex - 12 * (i + 1))

df_temp = {}
for tick in tickers:
    df_temp[tick] = df_dict[tick][indices[0]-1:lastIndex]
                            
df_dict_list = [df_temp]

for i in range(1, len(indices)):
    df_temp = {}
    for tick in tickers:
        df_temp[tick] = df_dict[tick][indices[i]-1:indices[i - 1]]
        
    df_dict_list.append(df_temp)

## Calculation of the Momenta
The following cell calculates the corresponding value differences and combines them into the Momentum number $\mu$.

In [ ]:
df_list = []
for df in df_dict_list:
    df_list.append(CalculateMomentum(df, weights = weights, MomentumTreshhold = 0, capital = capital))

## Exporting to Excel
Saves the calculated momenta, number of shares to buy and value of the shares into an XLSX file.

In [ ]:
for i in range(len(df_list)):
    df_list[i].round(2).to_excel(f"Data/QualityMomentum{2023-i}.xlsx")

## Graphing the Portfolio
Visualizes the breakdown of the resulting portfolio in a Pie chart, stocks which make up less than `remainderThreshhold` of the total Portfolio are combined into the "Remainder" category.

Stocks which make up less than `labelThreshold` of the total value don't have their percentage displayed.

In [ ]:
def PiPlot(df, remainderThreshold = 0.016, labelThreshold = 0.02):
    totalValue = df["Share Value"].sum()
    inefficiency = capital - totalValue

    selected = df[df["Share Value"] / totalValue > remainderThreshold]
    n = len(selected)
    remainder = totalValue - selected["Share Value"].sum()
    printList = np.append(df["Share Value"][:n], remainder)
    printLabel = np.append(df.index[:n].values, "Remainder")
    printExplode = np.append(np.zeros((1, n)), 0).tolist()

    plt.pie(printList, labels = printLabel, explode = printExplode, autopct = lambda x : f"{round(x, 1)}%" if x > labelThreshold * 100 else "", shadow = True)
    plt.show()

In [ ]:
print("2023")
PiPlot(df_list[0], remainderThreshold = 0.03, labelThreshold = 0.03)
print("2022")
PiPlot(df_list[1], remainderThreshold = 0.02, labelThreshold = 0.03)
print("2021")
PiPlot(df_list[2], remainderThreshold = 0.03, labelThreshold = 0.042)
print("2020")
PiPlot(df_list[3], remainderThreshold = 0.03, labelThreshold = 0.03)

## Backtesting
Picks a random time, calculates what stocks would have been bought and what the return would have been obtained if the strategy would have been followed. Of course this is theoretical because the companies in the SNP vary, here we just assume that the S&P500 is temporaly consistent.

Let's first go through everything with an example, we have the above Portfolio breakdown for 2021-2022, according to this we have the following portfolio breakdown:

In [ ]:
currVal_list = []
for df in df_list:
    curr = 0
    for tick in df.index:
        curr = curr + df["Shares"][tick] * df_dict[tick]["Open"][-1]
        
    currVal_list.append(curr)
    
for c in currVal_list:
    print(c)

In [ ]:
returns = []
startInvestment = []
for i in range(len(currVal_list)):
    startInvestment.append(df_list[i]["Share Value"].sum())
    returns.append(currVal_list[i] - df_list[i]["Share Value"].sum())

In [ ]:
print("Using the weights:")
for w in weights:
    print(w)
print("We obtain the following results")
print(f"On an initial capital of ${capital:.2f} the returns starting in the corresponding year would have been:")
for i in range(len(returns)):
    print(f"{2022 - i}: ${startInvestment[i]:.2f} invested, current value = ${currVal_list[i]:.2f} meaning a total profit of ${returns[i]:.2f}, which is a return of {(returns[i] / startInvestment[i]) * 100 / (i + 1):.2f}%.")